## Single Hidden Layer Trial
### Compositional Lyapunov function

In [3]:
# imports 
import tensorflow as tf
import numpy as np
import pandas as pd
import tensorflow.keras.backend as kb

In [1]:
# define the vector field
# this is actually a form that gives a compositional lyapunov function
# it is equation 7.1 from the grune paper
def vf(x):
    y = [-x[:,0]-10.*x[:,1]**2,-2.*x[:,1]]
    return y


# # this is equation 29 from main paper
# def vf(x):
#     y = [-x[:,1], x[:,0] + (1-x[:,0]**2)*x[:,1]]
#     return y


# define the upper bound for the boundary condition
def upperbound(data):
    return 10.*data[:,0]**2 + 10.*data[:,1]**2

# define the lower bound for the boundary condition
def lowerbound(data):
    return 0.1*data[:,0]**2 + 0.1*data[:,1]**2

In [33]:
# create data
inputdim = 2
data = 2.*np.random.random((200000, inputdim)) - 1
ubound = upperbound(data)
lbound = lowerbound(data)
vfdata = vf(data)

tdata   = tf.convert_to_tensor(data,   dtype=tf.float32)
tubound = tf.convert_to_tensor(ubound, dtype=tf.float32)
tlbound = tf.convert_to_tensor(lbound, dtype=tf.float32)
tvf     = kb.transpose(tf.convert_to_tensor(vfdata, dtype=tf.float32))
tzeros  = kb.zeros(tubound.shape)

# usually we have x_train, y_train
train_dataset_raw = tf.data.Dataset.from_tensor_slices((tdata, tubound, tlbound, tvf, tzeros))

In [34]:
def shuffle_and_batch_dataset(dataset, batch_size, shuffle_buffer=None):
    """
    This function is used to shuffle and batch the dataset, using shuffle_buffer
    and batch_size.
    Your function should return the shuffled and batched Dataset.
    """
    if shuffle_buffer:
        dataset = dataset.shuffle(buffer_size=shuffle_buffer)
        
    dataset = dataset.batch(batch_size)

    return(dataset)

train_dataset = shuffle_and_batch_dataset(train_dataset_raw, 32, shuffle_buffer=1024)
train_dataset.element_spec

(TensorSpec(shape=(None, 2), dtype=tf.float32, name=None),
 TensorSpec(shape=(None,), dtype=tf.float32, name=None),
 TensorSpec(shape=(None,), dtype=tf.float32, name=None),
 TensorSpec(shape=(None, 2), dtype=tf.float32, name=None),
 TensorSpec(shape=(None,), dtype=tf.float32, name=None))

In [24]:
def get_compile_and_fit_args():
    """
    This function is used to create the optimizer, loss, metric and callback objects. 
    Each of these should be created as instances from the corresponding classes in the
    optimizers, losses and metrics modules respectively, with the options as above.
    The function should then return the tuple (optimizer, loss, metric, callback)
    """
    opt = tf.keras.optimizers.Adam(learning_rate=0.0005)
    # opt = tf.keras.optimizers.SGD(learning_rate=0.0005)
    # opt = 'sgd'
    # loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    # loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
    loss = tf.keras.losses.MeanSquaredError()
    # loss='sparse_categorical_crossentropy'
    # acc = tf.keras.metrics.BinaryAccuracy()
    # acc = tf.keras.metrics.Accuracy()
    acc = 'accuracy'
    earlystopping = tf.keras.callbacks.EarlyStopping(patience=200)
    return opt, loss, acc, earlystopping


def compile_and_fit(model, optimizer, loss, num_epochs, train_dataset, 
                    validation_dataset=None, metrics=None, callbacks=None):
    """
    This function should compile and fit the model according to the above specifications.
    It should then return the History object returned by the fit method
    """
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    history = model.fit(train_dataset, epochs=num_epochs, verbose=1,
                        validation_data=validation_dataset, callbacks=callbacks)
    return history

In [29]:
def get_regularised_bn_mlp(input_shape, hidden_units, l2_reg_coeff = None, dropout_rate = None):
    """
    This function is used to build the MLP model. It takes input_shape and hidden_units
    as arguments, which should be used to build the model as described above, using the
    functional API.
    Your function should return the model.
    """
    inputs = tf.keras.layers.Input(shape=input_shape, name = 'state')
    # h = inputs
    h = tf.keras.layers.Dense(32, activation='linear')(inputs)
    for units in hidden_units:
        # h = tf.keras.layers.Dense(units, activation='selu', 
        #                           kernel_regularizer=tf.keras.regularizers.l2(l2_reg_coeff))(h)
        h = tf.keras.layers.Dense(units, activation='linear')(h)
        # h = tf.keras.layers.BatchNormalization()(h)
        # h = tf.keras.layers.Dropout(dropout_rate)(h)
    # outputs = tf.keras.layers.Dense(10, activation=None)(h)
    outputs = tf.keras.layers.Dense(1, activation='linear', name = 'Lyapunov_function')(h)
    model = tf.keras.models.Model(inputs=inputs, outputs=outputs)
    return model

model = get_regularised_bn_mlp(input_shape=inputdim, hidden_units=[])
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 state (InputLayer)          [(None, 2)]               0         
                                                                 
 dense_3 (Dense)             (None, 32)                96        
                                                                 
 Lyapunov_function (Dense)   (None, 1)                 33        
                                                                 
Total params: 129
Trainable params: 129
Non-trainable params: 0
_________________________________________________________________


In [31]:
opt, bce_loss, bin_acc, early_stopping = get_compile_and_fit_args()

history = compile_and_fit(model, 
                    optimizer=opt, 
                    loss=bce_loss, 
                    num_epochs=15, 
                    train_dataset=train_dataset, 
                    metrics=[bin_acc])

Epoch 1/15


ValueError: in user code:

    File "/Users/lysi2/Documents/UNI_Imperial/Deep_Learning/DL/lib/python3.7/site-packages/keras/engine/training.py", line 878, in train_function  *
        return step_function(self, iterator)
    File "/Users/lysi2/Documents/UNI_Imperial/Deep_Learning/DL/lib/python3.7/site-packages/keras/engine/training.py", line 867, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/lysi2/Documents/UNI_Imperial/Deep_Learning/DL/lib/python3.7/site-packages/keras/engine/training.py", line 860, in run_step  **
        outputs = model.train_step(data)
    File "/Users/lysi2/Documents/UNI_Imperial/Deep_Learning/DL/lib/python3.7/site-packages/keras/engine/training.py", line 805, in train_step
        x, y, sample_weight = data_adapter.unpack_x_y_sample_weight(data)
    File "/Users/lysi2/Documents/UNI_Imperial/Deep_Learning/DL/lib/python3.7/site-packages/keras/engine/data_adapter.py", line 1581, in unpack_x_y_sample_wei